In [933]:
import pandas as pd
from pandas import DataFrame, read_csv, Series
import numpy as np
import math
import sklearn.datasets
from sklearn.cluster import KMeans
import nltk
import re
from sklearn import *
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [934]:
# получение данных о студентах и их оценках по дисциплинам
import csv
def read_file(filename):
    f = open(filename)
    data = csv.reader(f)
    mentions = dict()
    for line in data:
        student = line[0]
        discipline = line[6]
        type_d = line[7]
        sem_rate = int(line[8])
        bonus = int(line[9])
        exam_rate = int(line[10])
        rate = 0
        if type_d == 'exam':
            if (sem_rate >= 38) & (exam_rate >= 22):
                rate = sem_rate + exam_rate + bonus
        else: 
            if (sem_rate >= 60):
                rate = sem_rate + bonus
        if not student in mentions:
            mentions[student] = dict()
        mentions[student][discipline] = rate
    f.close()
    return mentions

In [935]:
# косинусная мера сходства: насколько два пользователя похожи друг на друга. 
def dist_cosine(vecA, vecB):
    def dot_product(vecA, vecB):
        d = 0.0
        for dim in vecA:
            if dim in vecB: 
                d += vecA[dim]*vecB[dim]
        return d
    a = dot_product(vecA,vecB)
    b = math.sqrt(dot_product(vecA,vecA))
    c = math.sqrt(dot_product(vecB,vecB))
    if (a == 0) or (b == 0) or (c == 0):
        return 0
    else:
        return  (a / (b * c))

In [936]:
def similar_students(studentID, studentsData, nBestStudents):
    matches = [(u, dist_cosine(studentsData[studentID], studentsData[u])) for u in studentsData.keys() if u != studentID]
    bestMatches = sorted(matches, key=lambda q:(q[1], q[0]), reverse=True)[:nBestStudents]
    print("Most correlated with studentID: " + str(studentID))
    for line in bestMatches:
        print("StudentID: " + str(line[0]) + "  Coeff: " + str(line[1])) 
    return [(x[0], x[1]) for x in bestMatches]

In [937]:
students_test = read_file("raw_data32.csv")
#df = DataFrame(students_test)
#df.columns.name = 'studentID'
#df.index.name = 'discipline'    
rec = similar_students('1', students_test, 10)


Most correlated with studentID: 1
StudentID: 9  Coeff: 0.7542152087381664
StudentID: 6  Coeff: 0.7248624347806895
StudentID: 7  Coeff: 0.7194365076928019
StudentID: 5  Coeff: 0.6837763094049188
StudentID: 4  Coeff: 0.5597650673256156
StudentID: 2  Coeff: 0.22271363971105235
StudentID: 3  Coeff: 0.06705017139158977
StudentID: 8  Coeff: 0


In [938]:
# Похожесть дисциплин
def read_subjects(filename):
    f = open(filename)
    data = csv.reader(f)
    subjects = dict()
    for line in data:
        ID = line[0]
        discipline = line[1]
        subjects[ID] = discipline
    f.close()
    return subjects

In [939]:
#sd = read_subjects("subjects.csv")

#nltk.download() 

stopwords = set(stopwords.words("russian"))
#print(stopwords)


In [940]:
def review_to_words(review_text):
    review_text = review_text.replace('ё','е')
    review_text = review_text.replace('Ё','Е')
    # 1. Удаляем все, кроме букв
    letters_only = re.sub("[^а-яА-Я]", " ", review_text) 
    # 2. Делаем все буквы строчными и создаем массив слов
    words = letters_only.lower().split()                             
    # 3. Удаляем незначащие слова
    meaningful_words = [w for w in words if not w in stopwords]
    # 4. Формируем текст, объединяя слова через пробел
    return(" ".join(meaningful_words))

In [941]:
d0 = pd.read_csv("subjects2.csv", encoding='utf-8')
d0.head(3)

,ID,Name,Abbr
0,1,Математическая логика и теория множеств,Мат. логика и ТМ
1,2,Математический анализ,Мат. Анализ
2,3,Алгебра и геометрия,Алг. и геом.


In [942]:
d1 = d0.copy()
d1['Name'] = d1['Name'].apply(lambda s: review_to_words(s))
d1.head(3)

,ID,Name,Abbr
0,1,математическая логика теория множеств,Мат. логика и ТМ
1,2,математический анализ,Мат. Анализ
2,3,алгебра геометрия,Алг. и геом.


In [943]:
#vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 50000, binary = False) 
#vectorized = vectorizer.fit_transform(d1['Name'])
##print(vectorized)

#num_samples, num_features = vectorized.shape
## print((num_samples, num_features)) # 918 дисциплин, 1123-мерный вектор признаков

#vocabulary = vectorizer.get_feature_names()
##print(vocabulary[20])

#num_clusters = 50
#km = KMeans(n_clusters=num_clusters, init='random', n_init=10, verbose=1, random_state=3)
#km.fit(vectorized)


In [944]:
#km.predict(vectorized)
#km.cluster_centers_

In [945]:
##print(km.cluster_centers_)
#print(km.labels_)

In [946]:
#d1['Name'][km.labels_ == 46]

In [947]:
# стемминг
from nltk.stem.snowball import SnowballStemmer
# print(SnowballStemmer("russian").stem("гениально"))  # проверка работы стемминга

In [948]:
rus_stemmer = SnowballStemmer('russian')

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (rus_stemmer.stem(w) for w in analyzer(doc))
    
vectorizer = StemmedCountVectorizer(min_df=1, analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = None, binary = False)
vectorized = vectorizer.fit_transform(d1['Name'])

In [949]:
num_samples, num_features = vectorized.shape
vocabulary = vectorizer.get_feature_names()

num_clusters = 500
km = KMeans(n_clusters=num_clusters, init='k-means++', n_init=13, verbose=1, random_state=3)
km.fit(vectorized)

Initialization complete
Iteration  0, inertia 547.000
Iteration  1, inertia 432.987
Iteration  2, inertia 429.167
Iteration  3, inertia 428.675
Converged at iteration 3: center shift 0.000000e+00 within tolerance 4.368989e-07
Initialization complete
Iteration  0, inertia 546.000
Iteration  1, inertia 437.737
Iteration  2, inertia 434.944
Iteration  3, inertia 434.094
Converged at iteration 3: center shift 0.000000e+00 within tolerance 4.368989e-07
Initialization complete
Iteration  0, inertia 547.000
Iteration  1, inertia 436.430
Iteration  2, inertia 434.610
Iteration  3, inertia 433.119
Converged at iteration 3: center shift 0.000000e+00 within tolerance 4.368989e-07
Initialization complete
Iteration  0, inertia 552.000
Iteration  1, inertia 421.217
Iteration  2, inertia 420.542
Converged at iteration 2: center shift 0.000000e+00 within tolerance 4.368989e-07
Initialization complete
Iteration  0, inertia 550.000
Iteration  1, inertia 428.477
Iteration  2, inertia 427.031
Converged at

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=500, n_init=13, n_jobs=1, precompute_distances='auto',
    random_state=3, tol=0.0001, verbose=1)

In [950]:
km.predict(vectorized)
km.cluster_centers_

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [951]:
print(km.labels_)

[ 74  22  18   9 248  24  41 368  15 477 357 493  41 263  48 491  41  41
  58  41  12  14 135   1 283 134 301 184  27  12 310 256  11  93 227 289
   5 302  22 109 212  41  12  22 392 116 132   9 138 252  24 261 306  46
   8  41  41 136  24 122 426   5 486 287 450 212 212 351 337 136 210  10
  41 395  41 480 282 248 238 132   9 209  95  41 248  21  34  21  95 223
 370 337 344 331 212  34  31 373 380 173 136  41 310  14 497  10  24  41
 136 442 134 136 366 107 131  41 263  58  15 429  41 333 435 320 295  77
 399 246 242 169 169 430 376  74  60 381  94 165 286 347 294 474 132  41
 177 177 469 211 100 171  41 316  54  98 179 419 485 416  43 157  41  41
 281 105  48  48  93 379 167 390 221  33 213   9 345 222 441  44 114  15
  41 488 452  17  53  83 385  83  44 137 268  17  17  41  41  41 499 321
  41  17 121 403 361  38 268 189  41 139 250 248 473 384 234 397  53  75
  41 145 124 387  36  38  79  92 113 292  11  11 440  84 233 304  41  51
 166 324  41 194  64  33 242 354 169 154 438  90  1

In [952]:
d1['Name'][km.labels_ == 24]  
    # 25 кластер "плохой" при init = random, num_kluster = 500, n_init = 13
    # *** 41 кластер "плохой" при init = k_means++, num_kluster = 500, n_init = 13
    # хорошое разбиение при init = k_means++, num_kluster = 700, n_init = 13, но очень много "одиночных" дисциплин
    # 23 кластер "плохой" при init = k_means++, num_kluster = 600, n_init = 13
    # 79 кластер "плохой" при init = k_means++, num_kluster = 600, n_init = 15

5      теоретическая практическая механика
50                  теоретическая механика
58                 вычислительная механика
106      теоретическая прикладная механика
256                    механика разрушений
717               теоретическая педагогика
Name: Name, dtype: object

In [953]:
import scipy as sp

def similar_disciplines(from_info, discipline):
    discipline = review_to_words(discipline)
    discipline_vec = vectorizer.transform([discipline])
    discipline_label = km.predict(discipline_vec)[0]
    return from_info[km.labels_ == discipline_label]

In [955]:
sim_dis_info = similar_disciplines(d1['Name'], "Теория графов")
print(sim_dis_info)

659    теория конечных графов
Name: Name, dtype: object
